In [2]:
import folium
from shapely import geometry as geo
import pandas as pd
import geopandas as gpd
import math
import requests
import pprint
import numpy as np
from matplotlib import cm
import polyline
from matplotlib.colors import to_hex

In [3]:
dataset = "instances/Vrp-Set-A/A/A-n32-k5.vrp"
solset = "instances/Vrp-Set-A/A/A-n32-k5.sol"

In [4]:
class latlong:
  def __init__(self, lat, long):
        self.lat = lat
        self.long = long

In [5]:
def distance_matrix(coords):
    
    url = 'http://router.project-osrm.org/table/v1/driving/'
    for c in range(len(coords)) :
        lng = str(coords[c].long)
        lat = str(coords[c].lat)
        if c == 0 :
            url = url+lng+','+lat
        else :
            url = url +';'+lng + ',' + lat
    
    url = url+'?annotations=distance'
    
    print(url)
    
    distance = requests.get(url).json()['distances'];
   
    return distance


In [6]:
def dist(a, b):
    url = 'http://router.project-osrm.org/route/v1/driving/'
    url = url + str(a.long) + ',' + str(a.lat) + ';' + str(b.long) + ',' + str(b.lat)
    url = url + '?skip_waypoints=true'
    distance = requests.get(url).json()['routes'][0]['distance'];
    return distance

In [7]:
# def nearest(coord):
#     n = 1
#     nearestPnt = None
#     while True:
#         url = 'http://router.project-osrm.org/nearest/v1/driving/'
#         url = url + str(coord.long) + ',' + str(coord.lat)
#         url = url + '?number=' + str(n)
#         points = requests.get(url).json()['waypoints']
#         for point in points:
#             if point['name'] :
#                 nearestPnt = point
#                 break;
#         if nearestPnt :
#             break;
#         else :
#             n = 2*n
#             if n > 1000:
#                 print("Couldn't find nearest point")
#                 break;
#     return nearestPnt

In [8]:
def pnt_to_addr(coord):
    url = 'https://nominatim.openstreetmap.org/reverse?lat=' + str(coord.lat) + '&lon=' + str(coord.long) + '&addressdetails=0&format=jsonv2&zoom=21'
    response = requests.get(url).json()
    return response

In [9]:
pnt_to_addr(latlong(12.991377,80.235246))

{'place_id': 109250717,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 27768553,
 'lat': '12.991469250000002',
 'lon': '80.2346045614274',
 'place_rank': 30,
 'category': 'building',
 'type': 'university',
 'importance': 0,
 'addresstype': 'building',
 'name': 'Department of Managament Studies (DoMs)',
 'display_name': 'Department of Managament Studies (DoMs), Delhi Avenue, CMWSSB Division 174, Ward 174, Zone 13 Adyar, Chennai, Chennai District, Tamil Nadu, 600001, India',
 'boundingbox': ['12.9912339', '12.9917045', '80.2343474', '80.2348617']}

In [10]:
def embed(fname):
    file = open(fname, 'r')
    f = file.readlines()
    name = f[0].split()[2]
    n = int(f[3].split()[2])
    coords = []
    lng = []
    lat = []
    for i in range(7, 7+n):
        [x, y] = map(float, f[i].split()[1:])
        lng.append(x)
        lat.append(y)
    min_lng = min(lng)
    min_lat = min(lat)
    max_lng = max(lng)
    max_lat = max(lat)
    
    rad_lng = (max_lng - min_lng)/2
    rad_lat = (max_lat - min_lat)/2
    
    for i in range(len(lng)):
        
        x = 80.236 + 0.02*(lng[i] - 50)/50
        y = 13.0544 + 0.02*(lat[i] - 50)/50
        coords.append(latlong(y, x))
    dist = distance_matrix(coords)
    mymap = folium.Map(zoom_start = 9)
    for coord in coords:
        mymap.add_child(folium.Marker(location = [coord.lat, coord.long]))
    mymap.fit_bounds(mymap.get_bounds())
    mymap.save('map.html')
    return dist, coords

In [11]:
dist, ncoords = embed(dataset)

http://router.project-osrm.org/table/v1/driving/80.2488,13.0648;80.2544,13.052;80.236,13.036399999999999;80.2356,13.0376;80.22120000000001,13.037199999999999;80.22760000000001,13.069999999999999;80.23920000000001,13.0464;80.2496,13.049999999999999;80.22160000000001,13.043999999999999;80.2168,13.049999999999999;80.2172,13.0672;80.218,13.0384;80.2552,13.0552;80.2496,13.0444;80.24040000000001,13.058;80.21640000000001,13.0604;80.2512,13.0548;80.25240000000001,13.0352;80.2236,13.0472;80.2532,13.035599999999999;80.236,13.0716;80.2552,13.04;80.218,13.0512;80.2328,13.037999999999998;80.24040000000001,13.059199999999999;80.2196,13.0732;80.248,13.0564;80.2388,13.062;80.2252,13.0404;80.224,13.062399999999998;80.25,13.058399999999999;80.2552,13.036399999999999?annotations=distance


In [12]:
dist

[[0,
  2493.9,
  4963.1,
  4811,
  6196.1,
  3725.8,
  3208.7,
  2324.5,
  5808.5,
  5326.5,
  5056.3,
  6386.5,
  2609.5,
  2736.3,
  2069.1,
  5824.2,
  1881.5,
  4709,
  5038.8,
  4697.8,
  2651.7,
  3347.4,
  5022.6,
  5050,
  2710,
  4577.7,
  1844.5,
  1610.1,
  5450,
  4250.5,
  1050.4,
  4657.8],
 [3586.9,
  0,
  3848.1,
  3707.9,
  5093,
  5052.7,
  2421,
  1289.6,
  4705.4,
  4995.4,
  6383.2,
  5283.4,
  1522.8,
  1613.7,
  2868,
  5908.3,
  794.7,
  3028.2,
  4930.8,
  3017,
  4988.8,
  1702.6,
  4691.6,
  3730.3,
  3508.9,
  5904.6,
  1513.4,
  3403.9,
  4346.9,
  5577.4,
  1706.8,
  2977],
 [5101.1,
  4146.6,
  0,
  135.1,
  2492.7,
  4983.3,
  1892.9,
  3075.4,
  2105.1,
  3303.3,
  6313.8,
  2683.1,
  4262.2,
  2815.4,
  3252.9,
  5529.4,
  3540.1,
  2843.3,
  2330.5,
  2963.6,
  5564.2,
  3590.6,
  3559.2,
  626.9,
  4084.3,
  5835.2,
  3643.1,
  3979.2,
  1746.6,
  4548.1,
  4446.3,
  2923.6],
 [5047,
  4011.5,
  735.6,
  0,
  2357.6,
  4848.3,
  1838.8,
  2940.3,
  1

In [13]:
for c in ncoords:
    print(c.lat, c.long)

13.0648 80.2488
13.052 80.2544
13.036399999999999 80.236
13.0376 80.2356
13.037199999999999 80.22120000000001
13.069999999999999 80.22760000000001
13.0464 80.23920000000001
13.049999999999999 80.2496
13.043999999999999 80.22160000000001
13.049999999999999 80.2168
13.0672 80.2172
13.0384 80.218
13.0552 80.2552
13.0444 80.2496
13.058 80.24040000000001
13.0604 80.21640000000001
13.0548 80.2512
13.0352 80.25240000000001
13.0472 80.2236
13.035599999999999 80.2532
13.0716 80.236
13.04 80.2552
13.0512 80.218
13.037999999999998 80.2328
13.059199999999999 80.24040000000001
13.0732 80.2196
13.0564 80.248
13.062 80.2388
13.0404 80.2252
13.062399999999998 80.224
13.058399999999999 80.25
13.036399999999999 80.2552


In [14]:
file = open(solset, 'r')
f = file.readlines()
routes = []
for line in f[:-1]:
    r = [0]
    for stop in line[:-1].split(sep=': ')[1].split(sep=' '):
        r.append(int(stop))
    r.append(0)
    routes.append(r)
print(routes)

[[0, 21, 31, 19, 17, 13, 7, 26, 0], [0, 12, 1, 16, 30, 0], [0, 27, 24, 0], [0, 29, 18, 8, 9, 22, 15, 10, 25, 5, 20, 0], [0, 14, 28, 11, 4, 23, 3, 2, 6, 0]]


In [15]:
routeurl = "https://map.project-osrm.org/?z=13&center=13.061921%2C80.254440"
for route in routes:
    for stop in route:
        routeurl += "&loc=" + str(ncoords[stop].lat) + "%2C" + str(ncoords[stop].long)
routeurl

'https://map.project-osrm.org/?z=13&center=13.061921%2C80.254440&loc=13.0648%2C80.2488&loc=13.04%2C80.2552&loc=13.036399999999999%2C80.2552&loc=13.035599999999999%2C80.2532&loc=13.0352%2C80.25240000000001&loc=13.0444%2C80.2496&loc=13.049999999999999%2C80.2496&loc=13.0564%2C80.248&loc=13.0648%2C80.2488&loc=13.0648%2C80.2488&loc=13.0552%2C80.2552&loc=13.052%2C80.2544&loc=13.0548%2C80.2512&loc=13.058399999999999%2C80.25&loc=13.0648%2C80.2488&loc=13.0648%2C80.2488&loc=13.062%2C80.2388&loc=13.059199999999999%2C80.24040000000001&loc=13.0648%2C80.2488&loc=13.0648%2C80.2488&loc=13.062399999999998%2C80.224&loc=13.0472%2C80.2236&loc=13.043999999999999%2C80.22160000000001&loc=13.049999999999999%2C80.2168&loc=13.0512%2C80.218&loc=13.0604%2C80.21640000000001&loc=13.0672%2C80.2172&loc=13.0732%2C80.2196&loc=13.069999999999999%2C80.22760000000001&loc=13.0716%2C80.236&loc=13.0648%2C80.2488&loc=13.0648%2C80.2488&loc=13.058%2C80.24040000000001&loc=13.0404%2C80.2252&loc=13.0384%2C80.218&loc=13.03719999999

In [16]:
def add_route(routeline, m, c):
    folium.PolyLine(
        routeline,
        weight=4,
        color=c,
        opacity=0.8
    ).add_to(m)
    
    return m
# folium.Marker(
#     location=route['start_point'],
#     icon=folium.Icon(icon='play', color='green')
# ).add_to(m)

# folium.Marker(
#     location=route['end_point'],
#     icon=folium.Icon(icon='stop', color='red')
# ).add_to(m)

In [19]:
m = folium.Map(start_zoom=5)
m.add_child(folium.Marker(location=[ncoords[0].lat, ncoords[0].long]))

colors = ['lightgreen', 'beige', 'lightblue', 'green', 'gray', 'cadetblue', 'darkblue', 'darkred', 'darkpurple', 'red', 'lightgray', 'white', 'orange', 'black', 'blue', 'pink', 'darkgreen', 'purple', 'lightred']
colormap = cm.Dark2.colors
i = 0
for route in routes:
    routeapi = "http://router.project-osrm.org/route/v1/driving/"
    for stop in route:
        routeapi += str(ncoords[stop].long) + "," + str(ncoords[stop].lat) + ";"
        folium.Circle(
            location=[ncoords[stop].lat,ncoords[stop].long],
            color=to_hex(colormap[i])
        ).add_to(m)
    routeapi = routeapi[:-1]
    routejson = requests.get(routeapi).json()["routes"]
    routeline = polyline.decode(routejson[0]['geometry'])
    add_route(routeline, m, to_hex(colormap[i]))
    i+=1
m.fit_bounds(m.get_bounds())
m.save("routes_chennai.html")

In [13]:
import numpy as np
ndist = np.array(dist)
ndist

array([[   0. , 2493.9, 4963.1, ..., 4250.5, 1050.4, 4657.8],
       [3586.9,    0. , 3848.1, ..., 5577.4, 1706.8, 2977. ],
       [5101.1, 4146.6,    0. , ..., 4548.1, 4446.3, 2923.6],
       ...,
       [3743.7, 6098.3, 4734.3, ...,    0. , 4654.9, 8262.3],
       [2542.8, 1854.5, 4323.6, ..., 5269.1,    0. , 4018.4],
       [6059.9, 2405.6, 3148.2, ..., 7111.6, 3721.4,    0. ]])

In [14]:
from math import radians, cos, sin, asin, sqrt
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371000
      
    # calculate the result
    return(c * r)

distance(13.0044, 13.1044, 80.186, 80.286)/math.sqrt(0.02)

109767.46466941554

In [15]:
file = open(dataset, 'r')
f = file.readlines()
name = f[0].split()[2]
n = int(f[3].split()[2])
orig = []
for i in range(7, 7+n):
    [x, y] = map(float, f[i].split()[1:])
    orig.append([x, y])

odist = np.zeros_like(ndist)
for i in range(len(orig)):
    for j in range(i, len(orig)):
        odist[i][j] = math.sqrt((orig[i][0] - orig[j][0])**2 + (orig[i][1] - orig[j][1])**2) * 109767.46466941554 * 0.02/50
print(*odist)

[   0.         1533.60508533 3419.39307034 3318.68274016 4284.4759879
 2396.05052305 2278.09144117 1626.93009658 3758.60212496 3870.04651867
 3478.64162166 4452.82649013 1266.47111118 2240.97746036 1186.30143211
 3589.11075508 1128.84510571 3273.05891762 3373.98832603 3241.39438723
 1590.98464314 2811.41885403 3695.75822337 3426.15188386 1108.16222228
 3335.19730513  926.21888156 1139.89165142 3726.14870669 2734.95060705
  714.75394586 3195.57202894] [   0.            0.         2647.92253189 2599.42706091 3989.98317079
 3543.70451359 1778.09740709  570.79081628 3705.91607548 4133.09124243
 4411.06808572 4265.31001232  362.06628087  986.68678973 1671.92821066
 4271.85865638  466.73766197 1857.11498753 3421.64749159 1804.99904695
 2950.92838509 1320.13346326 3996.50058616 2825.44098018 1728.06205299
 4472.91306959  852.51945715 2033.98845463 3448.86509265 3526.79977152
  852.51945715 1714.62261072] [   0.            0.            0.          138.84608053 1626.93009658
 3801.69594914 115

In [16]:
diff = 0
for i in range(len(orig)):
    for j in range(i, len(orig)):
        diff += (((ndist[i][j]+ndist[j][i])/2) - odist[i][j]*1.5)
mdiff = diff/(len(orig)*(len(orig)-1)/2)
mdiff

45.42509194456201

In [17]:
sqdiff = 0
for i in range(len(orig)):
    for j in range(i, len(orig)):
        sqdiff += (((ndist[i][j]+ndist[j][i])/2) - odist[i][j]*1.5)**2
rmsdiff = math.sqrt(sqdiff/(len(orig)*(len(orig)-1)/2))
rmsdiff

498.93843526226016

In [18]:
factor = np.zeros_like(ndist)
for i in range(len(orig)):
    for j in range(i+1, len(orig)):
        factor[i][j] = ((ndist[i][j]+ndist[j][i])/2) / (odist[i][j]*1.5)
print(*factor)

[0.         1.3216788  0.98109029 0.99015189 0.96827088 0.96440649
 0.94795727 1.22697343 1.02932771 0.96052248 0.92041675 0.96016167
 1.63867404 1.04686461 1.25305421 1.0347317  1.47767542 1.06480413
 1.01065357 1.14556666 1.03566891 1.02872137 0.93767858 0.99112166
 1.51220339 0.87203237 1.71075473 1.00073839 1.03334577 0.97432594
 1.67572819 1.11797407] [0.         0.         1.00641162 0.9898848  0.86107297 0.99838083
 0.92203423 1.53640641 0.8516023  0.83746841 1.00406521 0.83524214
 1.87645201 1.26203507 1.15660867 0.91757093 1.24095407 1.04588749
 0.97248475 1.07196733 1.08834676 0.99830815 0.80308592 0.92398556
 1.22956619 0.92458016 1.2592479  1.10041595 0.90972921 1.10352168
 1.39246089 1.04641102] [0.         0.         0.         2.09032428 1.02467013 0.93256625
 1.08100656 0.97287894 0.93144774 0.97726409 1.1550611  0.93544179
 0.91988073 1.06243655 0.96893405 1.1772642  0.99822784 1.05939114
 1.01735972 1.14526587 0.988203   1.0110861  1.03623943 1.09740027
 0.9903061  0.

In [19]:
fsum = 0
for i in range(len(orig)):
    for j in range(i, len(orig)):
        fsum += factor[i][j]
fmean = fsum/(len(orig)*(len(orig)-1)/2)
fmean

1.060936575818164

In [20]:
dsum = 0
for i in range(len(orig)):
    for j in range(i, len(orig)):
        dsum += odist[i][j]*1.5
dmean = dsum/(len(orig)*(len(orig)-1)/2)
dmean

3868.4039403135034

In [21]:
100*rmsdiff/dmean

12.89778531302564

In [22]:
geo1 = gpd.points_from_xy(df1.longitude, df1.latitude)
geo_df = gpd.GeoDataFrame(df1[['id']], geometry=geo1)

NameError: name 'df1' is not defined

In [ ]:
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry ]
map = folium.Map(zoom_start = 9)
# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    map.add_child(folium.Marker(location = coordinates))
    i = i + 1
map.fit_bounds(map.get_bounds())

In [ ]:
chennai_convex = {"type":"MultiPolygon","coordinates":[[[[80.215,13.119],[80.286,13.132],[80.259,12.992],[80.222,12.981],[80.198,13.048]]]]}

In [ ]:
poly = geo.shape(chennai_convex)

In [ ]:
# pnt = geo.Point(80.270186,13.0836939)
# poly.contains(pnt)
# folium.Circle(location=[13.0836939,80.270186], radius=5, color="red").add_to(mymap)

In [ ]:
map = folium.Map(zoom_start = 9)

In [ ]:
folium.GeoJson(poly, name="poly").add_to(map)
map.fit_bounds(map.get_bounds())
map

In [ ]:
import folium

m = folium.Map(location=(13.0,80.15))
lat_interval = 1
lon_interval = 1

for lat in range(1280, 1320, lat_interval):
     folium.PolyLine([[lat/100, 79.9],[lat/100, 80.4]], weight=0.5).add_to(m)

for lon in range(7990, 8040, lon_interval):
    folium.PolyLine([[12.8, lon/100],[13.2, lon/100]], weight=0.5).add_to(m)

m.fit_bounds(m.get_bounds())
m.save('a.html')

In [ ]:

import selenium.webdriver
import time
driver = selenium.webdriver.Firefox()
driver.set_window_size(4000, 3000)  # choose a resolution
driver.get('file:///home/madhav/Desktop/ugrc/mapping/a.html')
# You may need to add time.sleep(seconds) here
time.sleep(1)
driver.save_screenshot('a.png')

True